In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range
from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [4]:
BUY = 2
SELL = 0
NONE = 1

In [5]:
def labeling(df):
    df['SIGNAL_LABEL'] = NONE
    df['TP'] = 0
    df['SL'] = 0
    
    for i in tqdm(range(1,len(df)) ):
        for j in range(i,len(df)):
            if( df.low[j] <= df.low[i-1] ):
                break
            else:
                if(df.high[j] > 2*(df.close[i] - df.low[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = BUY
                    break


        for j in range(i,len(df)):
            if( df.high[j] >= df.high[i-1] ):
                break
            else:
                if(df.low[j] < 2*(df.close[i] - df.high[i-1] ) + df.close[i]):
                    df.SIGNAL_LABEL[i] = SELL
                    break

        
    return df

In [6]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [7]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

0.00010000000000021103

In [ ]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

In [8]:
SLIT_INDEX = int(0.8*len(df_an))
df_train = df_an.iloc[:SLIT_INDEX]
df_train.reset_index(drop=True, inplace=True)
df_test = df_an.iloc[SLIT_INDEX:]
df_test.reset_index(drop=True, inplace=True)

In [9]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [10]:
df_train = featureExtraction(df_train)
df_train = labeling(df_train)
df_test = featureExtraction(df_test)
df_test = labeling(df_test)


C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_h'].iloc[i] = last_pivot_h
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_l'].iloc[i] = last_pivot_l
  0%|                                                                                        | 0/15917 [00:00<?, ?it/s]C:\Users\meysam\AppData\Local\Temp\ipykernel_3120\4124363782.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [ ]:
df_train.columns

In [ ]:
df_train.pivots_h

In [ ]:
len(df_test)

In [16]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level',
    'MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
        'Ten_kij_cross','Cloud_cross',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [37]:
class RNN(nn.Module):
    def __init__(self, batch_size, input_size, hidden_dim,lstm_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.LSTM_layers = lstm_layers
    
        self.dropout = nn.Dropout(0.5)
        self.rnn = nn.GRU(input_size=input_size, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
        self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=self.hidden_dim)
        self.fc2 = nn.Linear(in_features=self.hidden_dim, out_features=3)
#         self.fc2 = nn.Linear(257, 1)

    def forward(self, x):
        h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
#         c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
        torch.nn.init.xavier_normal_(h)
#         torch.nn.init.xavier_normal_(c)
        out, _ = self.rnn(x, h)
        out = self.dropout(out)
        out = torch.relu_(self.fc1(out[:,-1,:]) )
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [13]:
class dataset(Dataset):
    def __init__(self,df,sequence_lenght,feature_col):
        self.df = df
        self.sequence_lenght = sequence_lenght
        self.feature_col = feature_col
        self.X = []
        self.Y = [] 
        self.indeces = []
        print("preparing data..")
        self.prepare_data()
        
    def prepare_data(self):
        for i in tqdm(range(self.sequence_lenght+1,len(self.df)) ):
#             if self.df.pivots_h[i-1] or self.df.pivots_l[i-1]:
            X_feat = [] 
            for j in range(self.sequence_lenght):
                feature = []
                feature.append(self.df.pivots_h[i-self.sequence_lenght+j])
                feature.append(self.df.pivots_l[i-self.sequence_lenght+j])
                for fc in self.feature_col:
                    feature.append(self.df[fc][i-self.sequence_lenght+j+1])

                X_feat.append(feature)  
            self.X.append(np.array(X_feat))# TODO: use torch tensor 
            self.Y.append(self.df.SIGNAL_LABEL[i])
            self.indeces.append(i)
                
    def get_feature_len(self):
        return self.X[0].shape[1]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self,index):
        return torch.from_numpy(self.X[index]).float(),self.Y[index]

In [14]:
epoch = 1000
batch_size = 600
sequence_lenght = 20

In [15]:
ds_train = dataset(df_train,sequence_lenght,feature_col)
ds_test = dataset(df_test,sequence_lenght,feature_col) 

loader_train = DataLoader(ds_train, batch_size=batch_size)
loader_test = DataLoader(ds_test, batch_size=batch_size)

preparing data..


100%|███████████████████████████████████████████████████████████████████████████| 15897/15897 [02:06<00:00, 126.04it/s]


preparing data..


100%|█████████████████████████████████████████████████████████████████████████████| 3901/3901 [00:29<00:00, 131.44it/s]


In [17]:
print(len(loader_train))
print(len(loader_test))
print(len(ds_train))
print(len(ds_test))

27
7
15897
3901


In [31]:
input_size = ds_train.get_feature_len() 
hidden_dim = 60
lstm_layers = 2

In [38]:
model = RNN(batch_size, input_size, hidden_dim,lstm_layers)

In [39]:
weight = torch.Tensor([0.4,0.2,0.4])

In [40]:
criterion = nn.CrossEntropyLoss(weight=weight)

In [41]:
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [42]:
model.train()
train_loss = []
test_loss = []
test_accuracy = []
for i in range(epoch):
    loss_epoch_train = 0
    for x_batch, y_batch in tqdm(loader_train):
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch_train += loss.detach().item()
        
    
    loss_epoch_test = 0
    model.eval()
    pred_test = []
    all_label = []
    
    for x_batch, y_batch in tqdm(loader_test):
        pred = model(x_batch)
        loss = criterion(pred,y_batch)
        loss_epoch_test += loss.detach().item()
        pred_test += list(pred.softmax(dim=1).argmax(dim=1).detach().numpy())
        all_label += list(y_batch.detach().numpy())
        
    train_loss.append(loss_epoch_train/len(loader_train))
    test_loss.append(loss_epoch_test/len(loader_test))
    pred_test = np.array(pred_test)
    all_label = np.array(all_label)
    conf = confusion_matrix(all_label, pred_test)
    print(conf)
    test_accuracy.append((pred_test==all_label).sum()/len(all_label))
    print(train_loss[-1],test_accuracy[-1])

[[   0  765    0]
 [   0 2411    0]
 [   0  725    0]]
1.0708640769675926 0.6180466547039221
[[  17  748    0]
 [  45 2366    0]
 [   1  724    0]]
1.0587486205277619 0.6108690079466803
[[ 399  366    0]
 [ 837 1574    0]
 [  79  646    0]]
1.0160365877328095 0.5057677518584979
[[ 510  255    0]
 [ 917 1359  135]
 [  65  602   58]]
0.9478601018587748 0.4939759036144578
[[ 603  162    0]
 [1042 1266  103]
 [ 110  565   50]]
0.9052211708492703 0.49192514739810306
[[ 588  175    2]
 [ 983 1325  103]
 [ 123  550   52]]
0.8930945793787638 0.503716995642143
[[ 546  203   16]
 [ 858 1365  188]
 [ 140  485  100]]
0.8808413898503339 0.515508843886183
[[ 553  183   29]
 [ 863 1344  204]
 [ 146  460  119]]
0.8646795396451596 0.5167905665214048
[[ 551  179   35]
 [ 838 1344  229]
 [ 155  416  154]]
0.8485812412367927 0.5252499359138683
[[ 530  201   34]
 [ 802 1389  220]
 [ 146  413  166]]
0.8359724857189037 0.5344783388874648
[[ 463  267   35]
 [ 669 1540  202]
 [ 136  418  171]]
0.82735154584602

[[ 307  386   72]
 [ 511 1568  332]
 [ 111  381  233]]
0.2937795558461436 0.5403742630094848
[[ 303  392   70]
 [ 502 1545  364]
 [ 105  376  244]]
0.2977863163859756 0.5362727505767751
[[ 301  404   60]
 [ 467 1618  326]
 [  98  399  228]]
0.2839957552927512 0.5503716995642143
[[ 284  417   64]
 [ 448 1641  322]
 [ 104  395  226]]
0.26653458095259136 0.5513970776723917
[[ 288  411   66]
 [ 438 1651  322]
 [ 105  404  216]]
0.2596991081480627 0.552422455780569
[[ 284  415   66]
 [ 442 1654  315]
 [ 105  399  221]]
0.25436905329978027 0.5534478338887465
[[ 261  439   65]
 [ 410 1681  320]
 [ 102  408  215]]
0.2466832929187351 0.5529351448346578
[[ 235  464   66]
 [ 360 1739  312]
 [  99  422  204]]
0.24242292703301818 0.558318379902589
[[ 249  450   66]
 [ 375 1744  292]
 [ 109  419  197]]
0.2460825299775159 0.5613945142271213
[[ 258  446   61]
 [ 396 1720  295]
 [ 107  415  203]]
0.2474873661994934 0.5590874134837222
[[ 219  483   63]
 [ 333 1751  327]
 [  82  409  234]]
0.246594524769

KeyboardInterrupt: 

In [ ]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]-1

df_test.reset_index(drop=True, inplace=True)

In [ ]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

In [ ]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio =1# abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = 2#abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

In [ ]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])